In [3]:
import requests 
import time
import re
from datetime import datetime
from bs4 import BeautifulSoup,NavigableString, Comment

In [ ]:
def get_news_bongdaplus():
    _bongdaplus = {
            "home_page":"https://bongdaplus.vn/",
            "urls":{
                "hautruong":
                {
                 "url":"https://thethaovanhoa.vn/the-gioi-sao.htm#",
                 "sub-category":{  
                    0:{"name":"Thế giới sao",
                     "url":"https://thethaovanhoa.vn/the-gioi-sao.htm",
                     "cate_id":38,
                      "url_list" : []},
                 }
                }
            }
        }

In [6]:
url = "https://thethaovanhoa.vn/the-gioi-sao.htm"
response = requests.get(url)
time.sleep(2)

In [78]:
def convert_string(input_str):
    # Regular expression pattern to match a date in the format dd-mm-yyyy
    pattern = r'\b\d{2}/\d{2}/\d{4}\b'
    
    # Search for the pattern in the string
    match = re.search(pattern, input_str)
    if match:
    # Extract and print the date if found
        date_str = match.group(0)
        date_obj = datetime.strptime(date_str, '%d/%m/%Y')
        date =  date_obj.strftime('%Y-%m-%d')

        return date
    else:
        'No date found'
    
#convert time string to datetime object
def convert_time_string(posted_date):
    # Regular expression pattern to match a date in the format dd-mm-yyyy
    pattern = r'\b\d{2}/\d{2}/\d{4}\b'

    # Search for the pattern in the string and extract the date
    match = re.search(pattern, posted_date)
    if match:
        date_str = match.group(0)
        # Convert the extracted date string to datetime object
        date_obj = datetime.strptime(date_str, '%d/%m/%Y')
        return date_obj
    else:
        return None 
def get_list_url(cate_url): 
    response = requests.get(cate_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    post_content_div = soup.find('ul', class_ ="news-stream col550 left col547")
    list_url = post_content_div.find_all('li')
    urls = []
    for i in list_url:
        path = i.find('a',class_='img left show-popup visit-popup')
    
        try:
            url = 'https://thethaovanhoa.vn' + path['href']
            urls.append(url)
        except TypeError as e:
            continue
    return urls


In [46]:
test_url = get_list_url('https://thethaovanhoa.vn/the-gioi-sao.htm')

In [47]:
test_url[0]

'https://thethaovanhoa.vn/cau-thu-viet-huong-luong-cao-nhat-v-league-mo-tai-khoan-tai-nam-dinh-trung-ve-duy-manh-khang-dinh-thoi-toi-roi-20231127100957721.htm'

In [112]:
response = requests.get(test_url[0])
soup = BeautifulSoup(response.content, 'html.parser')
wrapper = soup.find('div', class_ = "col780 left clearafter")
h1 = wrapper.find('h1').text.strip()
date_str = wrapper.find('p',class_ ='news-time left').text
published_date = convert_string(date_str)
article = soup.find('div', class_ ="entry-body normal clearafter")
bottom_div = article.find_all(recursive = True)[-4:]
for i in bottom_div:
    i.decompose()
preview_div = article.find('div', class_ ='VCSortableInPreviewMode alignCenter type-7_preview')
preview_div.decompose()

In [125]:
print(article.prettify())

<div class="entry-body normal clearafter" data-role="content">
 <p>
  <b>
   Tiền đạo Văn Toàn đã có bàn thắng đầu tiên cho CLB Thép Xanh Nam Định sau khi vào sân từ ghế dự bị. Trên trang cá nhân, ngôi sao của ĐT Việt Nam đã ăn mừng chiến tích mới của mình và nhận được nhiều lời chúc mừng từ các đồng nghiệp cũng như người hâm mộ.
  </b>
 </p>
 <!-- bonewsrelation -->
 <h2>
  Cầu thủ Việt hưởng lương cao nhất V-League 'mở tài khoản' tại Nam Định
 </h2>
 <p>
  Hành quân đến sân của Bình Phước ở trận ra quân Cúp Quốc gia, CLB Nam Định phải căng mình chống đỡ trong phần lớn thời gian của hiệp 1. Tuy nhiên đến phút 38, tiền đạo Trần Văn Đạt bất ngờ mở tỷ số cho đội khách sau pha dứt điểm cận thành.
 </p>
 <p>
  Văn Toàn vào sân sau giờ nghỉ. Sự bổ sung trên hàng công giúp Nam Định chiếm lĩnh thế trận. Ngôi sao tấn công của ĐT Việt Nam chỉ mất 2 phút để tỏa sáng nhờ pha xoay sở khéo léo trong vòng vây 3 hậu vệ Bình Phước, trước khi mang về bàn nhân đôi cách biệt.
 </p>
 <figure class="VCSort

In [132]:
figures = article.find_all('figure', class_ = "VCSortableInPreviewMode")
for figure in figures:
    img = figure.find('img')
    caption_element = figure.find('figcaption', class_="PhotoCMS_Caption")
    caption = caption_element.find('p').text.strip()
    caption_start = NavigableString("[caption id=\"\" align=\"aligncenter\" width=\"800\"]")
    caption_text = NavigableString(caption)
    caption_end = NavigableString("[/caption]")
    #caption_text_list[i].decompose()
    # Insert the custom tags and caption text around the <img> tag
    img.insert_before(caption_start)
    img.insert_after(caption_end)
    try:
        img.insert_after(caption_text) 
    except IndexError as e:
        print(e)
    caption_element.decompose()

In [133]:
article

<div class="entry-body normal clearafter" data-role="content">
<p><b>Tiền đạo Văn Toàn đã có bàn thắng đầu tiên cho CLB Thép Xanh Nam Định sau khi vào sân từ ghế dự bị. Trên trang cá nhân, ngôi sao của ĐT Việt Nam đã ăn mừng chiến tích mới của mình và nhận được nhiều lời chúc mừng từ các đồng nghiệp cũng như người hâm mộ.</b></p><!-- bonewsrelation --><h2>Cầu thủ Việt hưởng lương cao nhất V-League 'mở tài khoản' tại Nam Định</h2><p>Hành quân đến sân của Bình Phước ở trận ra quân Cúp Quốc gia, CLB Nam Định phải căng mình chống đỡ trong phần lớn thời gian của hiệp 1. Tuy nhiên đến phút 38, tiền đạo Trần Văn Đạt bất ngờ mở tỷ số cho đội khách sau pha dứt điểm cận thành.</p><p>Văn Toàn vào sân sau giờ nghỉ. Sự bổ sung trên hàng công giúp Nam Định chiếm lĩnh thế trận. Ngôi sao tấn công của ĐT Việt Nam chỉ mất 2 phút để tỏa sáng nhờ pha xoay sở khéo léo trong vòng vây 3 hậu vệ Bình Phước, trước khi mang về bàn nhân đôi cách biệt.</p><figure class="VCSortableInPreviewMode small-img" style="" 